# Betfair Trading Data Analysis

This notebook demonstrates how to fetch and visualize Betfair market trading data using F# and Plotly.NET.

## Features:
- Load trading data from local Betfair API
- Process price and volume time series data
- Create interactive charts with dual y-axes
- Calculate trading statistics

## 1. Setup and Package Installation

First, let's install the required NuGet packages for HTTP requests, JSON parsing, and plotting.

In [28]:
#r "nuget: FSharp.Data"
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: Newtonsoft.Json"

Installed Packages FSharp.Data, 6.6.0 Newtonsoft.Json, 13.0.3 Plotly.NET, 4.2.0 Plotly.NET.Interactive, 4.2.0

## 2. Import Required Libraries

Import the necessary libraries for data processing, HTTP requests, and visualization.

In [30]:
open System
open System.Net.Http
open System.Threading.Tasks
open FSharp.Data
open Plotly.NET
open Plotly.NET.LayoutObjects
open Newtonsoft.Json
open Newtonsoft.Json.Linq

// Enable Plotly.NET interactive display
Plotly.NET.Defaults.DefaultDisplayOptions <- DisplayOptions.init(PlotlyJSReference = PlotlyJSReference.CDN "latest")

## 3. Define Data Models

Define F# record types to represent our trading data structure.

In [31]:
// Data models for Betfair trading data
type TradingEntry = {
    Time: DateTime
    Price: decimal
    Volume: decimal
}

type Selection = {
    Name: string
    Price: decimal
}

type Market = {
    EventName: string
    MarketName: string
    StartTime: DateTime
}

type TradingData = {
    Market: Market
    Selection: Selection
    TradedPricesAndVolume: TradingEntry list
}

// Aggregated data point for time series
type TimeSeriesPoint = {
    Time: DateTime
    Price: decimal
    Volume: decimal
}

## 4. API Configuration

Set up the API endpoint and parameters for fetching trading data.

In [32]:
// API configuration
let baseUrl = "http://localhost:10043"
let dataContextName = "MarketSelectionsPriceHistoryData"
let marketId = "1.244137767"
let selectionId = "500893_0.00"

let apiUrl = 
    $"{baseUrl}/api/getDataContextForBetfairMarketSelection?dataContextName={dataContextName}&marketId={marketId}&selectionId={selectionId}"

printfn "API URL: %s" apiUrl

API URL: http://localhost:10043/api/getDataContextForBetfairMarketSelection?dataContextName=MarketSelectionsPriceHistoryData&marketId=1.244137767&selectionId=500893_0.00http://localhost:10043/api/getDataContextForBetfairMarketSelection?dataContextName=MarketSelectionsPriceHistoryData&marketId=1.244137767&selectionId=500893_0.00



## 5. Data Fetching Functions

Create functions to fetch and parse the trading data from the API.

In [33]:
// HTTP client for API requests
let httpClient = new HttpClient()

// Function to fetch data from API
let fetchTradingDataAsync (url: string) = async {
    try
        let! response = httpClient.GetStringAsync(url) |> Async.AwaitTask
        return Ok response
    with
    | ex -> return Error $"Failed to fetch data: {ex.Message}"
}

// Function to parse JSON response
let parseTradingData (jsonString: string) =
    try
        let json = JObject.Parse(jsonString)
        
        // Check for API errors
        let isError = json.Value<bool>("isError") // Returns false if "isError" is missing, null, or not a bool
        if isError then
            let messageString = json.Value<string>("message") // Returns null if "message" is missing or null
            let messageOption = if String.IsNullOrWhiteSpace messageString then None else Some messageString
            let message = defaultArg messageOption "Unknown error"
            Error $"API Error: {message}"
        else
            // Parse market information
            let marketJson = json.["market"]
            let market = {
                EventName = marketJson.["eventName"].Value<string>()
                MarketName = marketJson.["marketName"].Value<string>()
                StartTime = marketJson.["startTime"].Value<DateTime>()
            }
            
            // Parse selection information
            let selectionsJson = json.["selections"] :?> JArray
            let selectionJson = selectionsJson.[0]
            let selection = {
                Name = selectionJson.["selection"].["name"].Value<string>()
                Price = selectionJson.["selection"].["price"].Value<decimal>()
            }
            
            // Parse trading data
            let tradingJson = selectionJson.["tradedPricesAndVolume"] :?> JArray
            let tradingEntries = 
                tradingJson
                |> Seq.map (fun item -> {
                    TradingEntry.Time = item.["time"].Value<DateTime>()
                    TradingEntry.Price = item.["price"].Value<decimal>()
                    TradingEntry.Volume = item.["volume"].Value<decimal>()
                }) // Record type TradingEntry is inferred
                |> List.ofSeq
            
            let tradingData = {
                Market = market
                Selection = selection
                TradedPricesAndVolume = tradingEntries
            }
            
            Ok tradingData
    with
    | ex -> Error $"Failed to parse JSON: {ex.Message}"

## 6. Data Processing Functions

Create functions to aggregate and process the trading data for visualization.

In [34]:
// Function to aggregate trading data by time
let aggregateByTime (entries: TradingEntry list) =
    entries
    |> List.groupBy (fun e -> e.Time)
    |> List.map (fun (time, group) -> {
        Time = time
        Price = group |> List.last |> fun e -> e.Price  // Use latest price for each time
        Volume = group |> List.sumBy (fun e -> e.Volume) // Sum volume for each time
    })
    |> List.sortBy (fun p -> p.Time)

// Function to calculate trading statistics
let calculateStats (points: TimeSeriesPoint list) =
    if points.IsEmpty then
        None
    else
        let prices = points |> List.map (fun p -> p.Price)
        let volumes = points |> List.map (fun p -> p.Volume)
        
        let minPrice = List.min prices
        let maxPrice = List.max prices
        let totalVolume = List.sum volumes
        let priceRange = if minPrice > 0m then (maxPrice - minPrice) / minPrice * 100m else 0m
        
        Some {|
            MinPrice = minPrice
            MaxPrice = maxPrice
            TotalVolume = totalVolume
            PriceRangePercent = priceRange
            DataPoints = points.Length
        |}

// Display statistics
let displayStats (stats: {| MinPrice : decimal; MaxPrice: decimal; TotalVolume: decimal; PriceRangePercent: decimal; DataPoints: int |} option) =
    match stats with
    | None -> printfn "No data available for statistics"
    | Some s -> 
        printfn "📊 Trading Statistics:"
        printfn "   💰 Total Volume: £%.2f" (float s.TotalVolume)
        printfn "   📈 Price Range: £%.2f - £%.2f" (float s.MinPrice) (float s.MaxPrice)
        printfn "   📊 Price Movement: %.2f%%" (float s.PriceRangePercent)
        printfn "   📍 Data Points: %d" s.DataPoints

## 7. Load and Process Trading Data

Fetch the data from the API and process it for visualization.

In [35]:
// Load trading data
let loadData () = async {
    printfn "🔄 Fetching trading data from API..."
    
    let! jsonResult = fetchTradingDataAsync apiUrl
    
    match jsonResult with
    | Error msg -> 
        printfn "❌ %s" msg
        return None
    | Ok jsonString ->
        printfn "📥 Data received, parsing..."
        
        match parseTradingData jsonString with
        | Error msg -> 
            printfn "❌ %s" msg
            return None
        | Ok tradingData ->
            printfn "✅ Data parsed successfully!"
            printfn "🏇 Event: %s" tradingData.Market.EventName
            printfn "🎯 Market: %s" tradingData.Market.MarketName
            printfn "⭐ Selection: %s" tradingData.Selection.Name
            printfn "💵 Current Price: £%.2f" (float tradingData.Selection.Price)
            printfn "📊 Raw Trading Records: %d" tradingData.TradedPricesAndVolume.Length
            
            // Aggregate data by time
            let aggregatedData = aggregateByTime tradingData.TradedPricesAndVolume
            printfn "📈 Aggregated Data Points: %d" aggregatedData.Length
            
            // Calculate and display statistics
            let stats = calculateStats aggregatedData
            displayStats stats
            
            return Some (tradingData, aggregatedData)
}

## 8. Create Interactive Charts

Generate professional-looking charts using Plotly.NET with dual y-axes for price and volume.

In [ ]:
// Function to create trading chart
let createTradingChart (tradingData: TradingData) (timeSeriesData: TimeSeriesPoint list) =
    if timeSeriesData.IsEmpty then
        printfn "❌ No data available for charting"
        None
    else
        let times = timeSeriesData |> List.map (fun p -> p.Time)
        let prices = timeSeriesData |> List.map (fun p -> float p.Price)
        let volumes = timeSeriesData |> List.map (fun p -> float p.Volume)
        
        // Create price line chart
        let priceTrace = 
            Chart.Scatter(
                x = times,
                y = prices,
                mode = StyleParam.Mode.Lines_Markers,
                Name = "Price"
                // YAxis = StyleParam.SubPlotId.YAxis 1 // This was incorrect
            )
            // The following line caused a type error and has been removed.
            // priceTrace will use the default Y-axis (Y1), which is configured by Chart.withYAxisStyle.
            |> Chart.withLineStyle(Width = 3., Color = Color.fromHex "#2E86AB")
            |> Chart.withMarkerStyle(
                Size = 6, 
                Color = Color.fromHex "#2E86AB",
                Outline = Line.init(Width = 2., Color = Color.fromKeyword White)
            )
        // Chart.withTraceInfo call removed as Name is set in Scatter and YAxis is now part of Scatter
        
        // Create volume bar chart
        let volumeTrace = 
            Chart.Bar( // Changed from StackedArea to Bar, aligning with the comment and Plotly.NET API
                x = times,
                y = volumes,
                Name = "Volume",
                YAxis = StyleParam.SubPlotId.YAxis 2 // Assign Volume trace to the second Y-axis (Y2)
            )
            // The Chart.withTraceInfo call previously here was removed because YAxis is not a valid
            // parameter for it. The YAxis mapping is now correctly set in the Chart.StackedArea constructor.
            // The Name property is already set in Chart.StackedArea.
            |> Chart.withMarkerStyle(
                Color = Color.fromHex "#FF9800",
                Opacity = 0.7,
                Outline = Line.init(Width = 1., Color = Color.fromHex "#FF6B35")
            )
        
        // Combine charts with dual y-axes
        let combinedChart = 
            [priceTrace; volumeTrace]
            |> Chart.combine
            |> Chart.withTitle(
                Title.init(
                    Text = $"{tradingData.Market.EventName} - {tradingData.Selection.Name}",
                    Font = Font.init(Size = 20., Color = Color.fromHex "#2c3e50")
                )
            )
            |> Chart.withXAxisStyle(
                Title.init("Time", Font = Font.init(Size = 14., Color = Color.fromHex "#34495e")),
                ShowGrid = true,
                GridColor = Color.fromRGBA(0, 0, 0, 0.1)
            )
            |> Chart.withYAxisStyle(
                Title.init("Price (£)", Font = Font.init(Size = 14., Color = Color.fromHex "#2E86AB")),
                Side = StyleParam.Side.Left,
                ShowGrid = true,
                GridColor = Color.fromRGBA(46, 134, 171, 0.2),
                Id = StyleParam.SubPlotId.YAxis 1
            )
            |> Chart.withY2AxisStyle(
                Title.init("Volume (£)", Font = Font.init(Size = 14., Color = Color.fromHex "#FF6B35")),
                Side = StyleParam.Side.Right,
                Overlaying = StyleParam.LinearAxisId.Y,
                ShowGrid = false,
                Id = StyleParam.SubPlotId.YAxis 2
            )
            |> Chart.withSize(1200, 600)
            |> Chart.withConfig(
                Config.init(
                    Responsive = true,
                    DisplayModeBar = true
                )
            )
        
        Some combinedChart

// Execute data loading
let dataResult = loadData () |> Async.RunSynchronously

// Create and display chart if data is available
match dataResult with
| None -> printfn "❌ No data available for charting"
| Some (tradingData, timeSeriesData) ->
    match createTradingChart tradingData timeSeriesData with
    | None -> printfn "❌ Failed to create chart"
    | Some chart -> 
        printfn "📊 Displaying interactive trading chart..."
        chart

Error: input.fsx (32,13)-(37,14) typecheck error The member or object constructor 'Column' does not take 0 argument(s). An overload was found taking 20 arguments.
input.fsx (55,17)-(55,98) typecheck error Type constraint mismatch. The type 
    'Title'    
is not compatible with type
    'string'    

input.fsx (57,35)-(57,43) typecheck error The type 'Color' does not define the field, constructor or member 'fromRGBA'. Maybe you want one of the following:
   fromRGB
   fromARGB
   fromString
input.fsx (60,17)-(60,103) typecheck error Type constraint mismatch. The type 
    'Title'    
is not compatible with type
    'string'    

input.fsx (63,35)-(63,43) typecheck error The type 'Color' does not define the field, constructor or member 'fromRGBA'. Maybe you want one of the following:
   fromRGB
   fromARGB
   fromString
input.fsx (66,22)-(66,37) typecheck error The type 'Chart' does not define the field, constructor or member 'withY2AxisStyle'. Maybe you want one of the following

## 9. Additional Analysis Charts

Create additional visualizations for deeper analysis.

In [ ]:
// Volume Distribution Analysis
match dataResult with
| None -> printfn "❌ No data available for volume analysis"
| Some (tradingData, timeSeriesData) ->
    let volumes = timeSeriesData |> List.map (fun p -> float p.Volume)
    
    let volumeHistogram = 
        Chart.Histogram(
            X = volumes,
            NBinsX = 20
        )
        |> Chart.withTitle("Volume Distribution")
        |> Chart.withXAxisStyle(Title.init("Volume (£)"))
        |> Chart.withYAxisStyle(Title.init("Frequency"))
        |> Chart.withMarkerStyle(Color = Color.fromHex "#FF9800", Opacity = 0.7)
        |> Chart.withSize(800, 400)
    
    printfn "📊 Volume Distribution Analysis:"
    volumeHistogram

In [ ]:
// Price Movement Analysis
match dataResult with
| None -> printfn "❌ No data available for price analysis"
| Some (tradingData, timeSeriesData) ->
    if timeSeriesData.Length < 2 then
        printfn "❌ Insufficient data for price movement analysis"
    else
        // Calculate price changes
        let priceChanges = 
            timeSeriesData
            |> List.pairwise
            |> List.map (fun (prev, curr) -> 
                let change = curr.Price - prev.Price
                (curr.Time, float change)
            )
        
        let times, changes = List.unzip priceChanges
        
        let priceChangeChart = 
            Chart.Column(
                keys = times,
                values = changes
            )
            |> Chart.withTitle("Price Changes Over Time")
            |> Chart.withXAxisStyle(Title.init("Time"))
            |> Chart.withYAxisStyle(Title.init("Price Change (£)"))
            |> Chart.withMarkerStyle(
                Color = changes |> List.map (fun c -> if c >= 0.0 then Color.fromHex "#4CAF50" else Color.fromHex "#F44336"),
                Opacity = 0.8
            )
            |> Chart.withSize(800, 400)
        
        printfn "📊 Price Movement Analysis:"
        priceChangeChart

## 10. Summary and Export

Display final summary and provide options for data export.

In [ ]:
// Final summary
match dataResult with
| None -> printfn "❌ Analysis completed with no data"
| Some (tradingData, timeSeriesData) ->
    printfn "\n🎯 Analysis Summary:"
    printfn "===================="
    printfn "📅 Event: %s" tradingData.Market.EventName
    printfn "🎯 Market: %s" tradingData.Market.MarketName
    printfn "⭐ Selection: %s" tradingData.Selection.Name
    printfn "🕐 Start Time: %s" (tradingData.Market.StartTime.ToString("yyyy-MM-dd HH:mm:ss"))
    printfn "💵 Current Price: £%.2f" (float tradingData.Selection.Price)
    
    let stats = calculateStats timeSeriesData
    match stats with
    | Some s ->
        printfn "\n📊 Trading Performance:"
        printfn "💰 Total Volume Traded: £%.2f" (float s.TotalVolume)
        printfn "📈 Price Range: £%.2f - £%.2f" (float s.MinPrice) (float s.MaxPrice)
        printfn "📊 Price Volatility: %.2f%%" (float s.PriceRangePercent)
        printfn "📍 Data Granularity: %d time points" s.DataPoints
        
        // Trading insights
        let avgVolume = s.TotalVolume / decimal s.DataPoints
        printfn "\n💡 Insights:"
        printfn "📊 Average Volume per Time Point: £%.2f" (float avgVolume)
        
        if s.PriceRangePercent > 5m then
            printfn "⚠️  High volatility detected (>5%% price movement)"
        elif s.PriceRangePercent < 1m then
            printfn "📈 Stable pricing (low volatility)"
        else
            printfn "📊 Moderate price movement"
            
        if s.TotalVolume > 1000m then
            printfn "💰 High trading activity"
        elif s.TotalVolume < 100m then
            printfn "📉 Low trading activity"
        else
            printfn "📊 Moderate trading activity"
    | None -> printfn "❌ No statistical data available"
    
    printfn "\n✅ Analysis complete!"
    printfn "🔄 Refresh the API data and re-run cells to get latest information."

## 11. Advanced Analytics

Advanced analysis functions for deeper insights into trading patterns, similar to the HTML file functionality.

In [ ]:
// Advanced analytics functions

// Function to analyze detailed price movements
let analyzePriceMovements (data: TimeSeriesPoint list) =
    if data.Length < 2 then
        printfn "Not enough data for movement analysis"
        None
    else
        let movements = 
            data
            |> List.pairwise
            |> List.map (fun (prev, curr) ->
                let change = curr.Price - prev.Price
                let percentChange = if prev.Price > 0m then (change / prev.Price) * 100m else 0m
                (curr.Time, change, percentChange))
        
        let upMoves = movements |> List.filter (fun (_, change, _) -> change > 0m) |> List.length
        let downMoves = movements |> List.filter (fun (_, change, _) -> change < 0m) |> List.length
        let sidewaysMoves = movements |> List.filter (fun (_, change, _) -> change = 0m) |> List.length
        
        let maxIncrease = movements |> List.maxBy (fun (_, _, pct) -> pct)
        let maxDecrease = movements |> List.minBy (fun (_, _, pct) -> pct)
        
        printfn "\n📈 Price Movement Analysis:"
        printfn "├── Up movements: %d" upMoves
        printfn "├── Down movements: %d" downMoves
        printfn "├── Sideways movements: %d" sidewaysMoves
        printfn "├── Largest increase: %.2f%% at %A" (let (_, _, pct) = maxIncrease in pct) (let (time, _, _) = maxIncrease in time)
        printfn "└── Largest decrease: %.2f%% at %A" (let (_, _, pct) = maxDecrease in pct) (let (time, _, _) = maxDecrease in time)
        
        Some {
            UpMoves = upMoves
            DownMoves = downMoves
            SidewaysMoves = sidewaysMoves
            MaxIncrease = maxIncrease
            MaxDecrease = maxDecrease
        }

// Function to analyze volume patterns
let analyzeVolumePatterns (data: TimeSeriesPoint list) =
    if data.Length = 0 then
        printfn "No data for volume analysis"
        None
    else
        let totalVolume = data |> List.sumBy (_.Volume)
        let avgVolume = totalVolume / decimal data.Length
        let maxVolume = data |> List.maxBy (_.Volume)
        let minVolume = data |> List.minBy (_.Volume)
        
        let highVolumeThreshold = avgVolume * 2m
        let highVolumePeriods = data |> List.filter (fun d -> d.Volume > highVolumeThreshold) |> List.length
        
        let volumeVariance = 
            data 
            |> List.map (fun d -> (d.Volume - avgVolume) * (d.Volume - avgVolume))
            |> List.average
        let volumeStdDev = sqrt (float volumeVariance)
        
        printfn "\n📊 Volume Analysis:"
        printfn "├── Average volume: £%.2f" avgVolume
        printfn "├── Volume std deviation: £%.2f" volumeStdDev
        printfn "├── Peak volume: £%.2f at %A" maxVolume.Volume maxVolume.Time
        printfn "├── Minimum volume: £%.2f at %A" minVolume.Volume minVolume.Time
        printfn "└── High volume periods (>2x avg): %d" highVolumePeriods
        
        Some {
            TotalVolume = totalVolume
            AverageVolume = avgVolume
            StdDeviation = decimal volumeStdDev
            PeakVolume = maxVolume
            MinVolume = minVolume
            HighVolumePeriods = highVolumePeriods
        }

// Define types for analytics results
type MovementAnalysis = {
    UpMoves: int
    DownMoves: int
    SidewaysMoves: int
    MaxIncrease: DateTime * decimal * decimal
    MaxDecrease: DateTime * decimal * decimal
}

type VolumeAnalysis = {
    TotalVolume: decimal
    AverageVolume: decimal
    StdDeviation: decimal
    PeakVolume: TimeSeriesPoint
    MinVolume: TimeSeriesPoint
    HighVolumePeriods: int
}

In [ ]:
// Enhanced chart creation with professional styling
let createProfessionalTradingChart (tradingData: TradingData) (timeSeriesData: TimeSeriesPoint list) =
    
    let times = timeSeriesData |> List.map (_.Time)
    let prices = timeSeriesData |> List.map (fun x -> float x.Price)
    let volumes = timeSeriesData |> List.map (fun x -> float x.Volume)
    
    // Create price trace with spline smoothing
    let priceTrace = 
        Chart.Spline(
            x = times,
            y = prices,
            Name = "Price (£)",
            LineColor = Color.fromHex "#2E86AB",
            LineWidth = 3.0,
            ShowMarkers = true
        )
        |> Chart.withTraceInfo(YAxis = StyleParam.SubPlotId.YAxis 1)
        |> Chart.withMarkerStyle(
            Size = 8,
            Color = Color.fromHex "#2E86AB",
            Line = Line.init(Color = Color.fromKeyword White, Width = 2.0)
        )
    
    // Create volume trace with gradient effect
    let volumeTrace =
        Chart.Column(
            keys = times,
            values = volumes,
            Name = "Volume (£)",
            MarkerColor = Color.fromHex "#FF9800"
        )
        |> Chart.withTraceInfo(YAxis = StyleParam.SubPlotId.YAxis 2)
        |> Chart.withMarkerStyle(
            Opacity = 0.7,
            Line = Line.init(Color = Color.fromHex "#FF6B35", Width = 1.0)
        )
    
    // Combine charts with enhanced styling
    [priceTrace; volumeTrace]
    |> Chart.combine
    |> Chart.withTitle(
        sprintf "%s - %s" tradingData.Market.EventName tradingData.Selection.Name,
        TitleFont = Font.init(Size = 24, Color = Color.fromHex "#2c3e50", Family = StyleParam.FontFamily.Arial)
    )
    |> Chart.withXAxisStyle(
        Title = Title.init("Time", Font = Font.init(Size = 14, Color = Color.fromHex "#34495e")),
        ShowGrid = true,
        GridColor = Color.fromRGBA(0, 0, 0, 0.1),
        ZeroLine = false
    )
    |> Chart.withYAxisStyle(
        Title = Title.init("Price (£)", Font = Font.init(Size = 14, Color = Color.fromHex "#2E86AB")),
        Side = StyleParam.Side.Left,
        ShowGrid = true,
        GridColor = Color.fromRGBA(46, 134, 171, 0.2),
        ZeroLine = false
    )
    |> Chart.withYAxis(
        LinearAxis.init(
            Title = Title.init("Volume (£)", Font = Font.init(Size = 14, Color = Color.fromHex "#FF6B35")),
            Side = StyleParam.Side.Right,
            Overlaying = StyleParam.LinearAxisId.Y 1,
            ShowGrid = false,
            ZeroLine = false
        ),
        Id = StyleParam.SubPlotId.YAxis 2
    )
    |> Chart.withLayoutStyle(
        PaperBGColor = Color.fromHex "#fafafa",
        PlotBGColor = Color.fromKeyword White,
        ShowLegend = true,
        Legend = Legend.init(
            X = 0.02,
            Y = 0.98,
            BGColor = Color.fromRGBA(255, 255, 255, 0.9),
            BorderColor = Color.fromHex "#bdc3c7",
            BorderWidth = 1.0
        )
    )
    |> Chart.withConfig(
        Config.init(
            Responsive = true,
            DisplayModeBar = true,
            DisplayLogo = false,
            ModeBarButtonsToRemove = ["pan2d"; "lasso2d"; "select2d"]
        )
    )
    |> Chart.withSize(1000, 600)

In [ ]:
// Run comprehensive analysis with all new features
match dataResult with
| None -> printfn "❌ No data available for comprehensive analysis"
| Some (tradingData, timeSeriesData) ->
    printfn "\n🚀 Running Comprehensive Analysis..."
    printfn "=========================================\n"
    
    // Run advanced analytics
    let movementAnalysis = analyzePriceMovements timeSeriesData
    let volumeAnalysis = analyzeVolumePatterns timeSeriesData
    
    // Create enhanced professional chart
    printfn "\n📊 Creating professional trading chart..."
    let professionalChart = createProfessionalTradingChart tradingData timeSeriesData
    
    // Display the enhanced chart
    professionalChart
    
    // Trading insights based on analytics
    printfn "\n💡 Advanced Trading Insights:"
    printfn "============================="
    
    match movementAnalysis, volumeAnalysis with
    | Some mvmt, Some vol ->
        let totalMoves = mvmt.UpMoves + mvmt.DownMoves + mvmt.SidewaysMoves
        let upPercentage = (float mvmt.UpMoves / float totalMoves) * 100.0
        let downPercentage = (float mvmt.DownMoves / float totalMoves) * 100.0
        
        printfn "📈 Price Trend: %.1f%% up, %.1f%% down movements" upPercentage downPercentage
        
        if upPercentage > 60.0 then
            printfn "🎯 Strong upward trend detected"
        elif downPercentage > 60.0 then
            printfn "📉 Strong downward trend detected"
        else
            printfn "↔️  Sideways/consolidating market"
        
        if vol.HighVolumePeriods > timeSeriesData.Length / 4 then
            printfn "💥 High volatility period - active trading"
        else
            printfn "📊 Normal trading activity"
        
        let volumeConsistency = (vol.AverageVolume / vol.StdDeviation) 
        if volumeConsistency > 2m then
            printfn "✅ Consistent volume patterns"
        else
            printfn "⚠️  Irregular volume patterns"
            
    | _ -> printfn "❌ Could not complete advanced analysis"
    
    printfn "\n✅ Comprehensive analysis complete!"